In [4]:
import gdal
import rasterio as rio
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib import pyplot
from osgeo import gdal
import numpy as np
import requests
import json
import fiona
import geopy
import pyproj
import shapely
import osr
import os
import pygis
import richdem as rd
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
%matplotlib inline

In [ ]:
# Sample Address:
# Francis Wellesplein 1, 2018 Antwerpen (152074.52, 210653.91)
# Sint-Pietersvliet 3, 2000 Antwerpen
# Tavernierkaai 3, 2000 Antwerpen

In [13]:
address = input("Enter the Address as 'street_name', 'number', 'postcode', 'city' : ")

Enter the Address as 'street_name', 'number', 'postcode', 'city' : Tavernierkaai 3, 2000 Antwerpen


In [17]:
##---- Getting the lambert co-ordinates for a given location : given_loc -->(x,y)------##

def lam_lat_lon(address):
    while True: 
        try:
            req = requests.get(f"http://loc.geopunt.be/geolocation/location?q={address}&c=1",)
            x = req.json()["LocationResult"][0]["Location"]["X_Lambert72"]
            y = req.json()["LocationResult"][0]["Location"]["Y_Lambert72"]
            return x, y
            continue

        except IndexError: 
            print("Entered address is not in correct format, Please retry") 
            break

given_loc = lam_lat_lon(address)
#print(type(given_loc))
#print(given_loc)


##---Prepare dataframe for small tile tif file----##

out_path = 'D:/BeCode/Maps/Test/Tiled_DSM_Output/'

files = os.listdir(out_path)
namelist =[]
for f in files:    
    namelist.append(f)
    
df = pd.DataFrame(namelist)
df[['1','2','3','x_center','y_center']] = df[0].str.split('_',expand=True)
df['y_center'] = df['y_center'].str.replace('.tif','')
df['x_center'] = pd.to_numeric(df['x_center'], errors='ignore') #Str to int
df['y_center'] = pd.to_numeric(df['y_center'], errors='ignore') #Str to int

#print(df.head())
#print(df.shape)

##---Prepare geopandas dataframe for small tile tif file----##

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x_center, df.y_center))
print(gdf.head())

##---Finding the nearest tile center from geo data frame---##

def tile_finder():
    
    pts = gdf['geometry']
    pts =  pts.unary_union # convert to Shapely MultiPoint geometry
    pt = Point(lam_lat_lon(address))   # Tuple to Point conversion
    output = [o.wkt for o in nearest_points(pt, pts)]
        
    return output[1]

tile_finder()

##---Identify the tile name for the given_loc---------##

rawinfile = tile_finder()        # Tile center point 
#print(rawinfile)
sp = rawinfile.split(" ")
fn = (sp[1])[1:] + ".0_" + (sp[2][:-1]) + ".0"
tile_filename = [d for d in os.listdir(out_path) if fn in d]
#print(tile_filename)

tile_dsmfile = 'D:/BeCode/Maps/Test/Tiled_DSM_Output/' + str(tile_filename[0])
tile_dtmfile = 'D:/BeCode/Maps/Test/Tiled_DTM_Output/' + str(tile_filename[0])

##-----Finding the polygon for a given address-----##

import regex as re

address_regx = re.compile("^([A-z- ]+)\s(\d+),\s(\d+)\s([A-z]+)")
result = address_regx.search(address)  #Address from user
street = result.group(1)
nb = result.group(2)
pc = result.group(3)
city = result.group(4)

req1 = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()
objectId = req1["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]

req2 = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()
objectId = req2["gebouw"]["objectId"]

req3 = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()
polygon = [req3["geometriePolygoon"]["polygon"]]


##-------3D Ploting---------##

from rasterio.mask import mask

tile_data = rio.open(tile_dsmfile)

crop_img, crop_transform = mask(dataset=tile_data, shapes=polygon, crop=True, indexes=1, nodata=0, filled=True)
#print(crop_img)

fig = go.Figure(data=[go.Surface(z=crop_img)])
fig.update_layout(title='3D Plot', autosize=True)
fig.show()

                                         0  1      2          3  x_center  \
0      0_0_namestart_130500.0_217500.0.tif  0      0  namestart  130500.0   
1      0_0_namestart_145500.0_246500.0.tif  0      0  namestart  145500.0   
2      0_0_namestart_226500.0_226500.0.tif  0      0  namestart  226500.0   
3  0_10000_namestart_130500.0_207500.0.tif  0  10000  namestart  130500.0   
4   0_1000_namestart_130500.0_216500.0.tif  0   1000  namestart  130500.0   

   y_center                       geometry  
0  217500.0  POINT (130500.000 217500.000)  
1  246500.0  POINT (145500.000 246500.000)  
2  226500.0  POINT (226500.000 226500.000)  
3  207500.0  POINT (130500.000 207500.000)  
4  216500.0  POINT (130500.000 216500.000)  
